In [22]:
import numpy as np
import networkx as nx
import generator

def calculate_distance_matrix(G):
    A= nx.adjacency_matrix(G).toarray()
    n_vertices=G.number_of_nodes()
    distance_matrix = np.zeros((n_vertices, n_vertices))


    for i in range(n_vertices):
        for j in range(i + 1, n_vertices):
            A_copy=A.copy()
            euclidean_matrix = (A_copy - A_copy[i,j])**2
            distance_matrix[i, j] = distance_matrix[j, i] = np.sqrt(euclidean_matrix.sum(axis=1)-A_copy[i,j]**2)[i]
    
    return distance_matrix

def find_closest_clusters(distance_matrix):
    np.fill_diagonal(distance_matrix, np.inf)
    return np.unravel_index(np.argmin(distance_matrix, axis=None), distance_matrix.shape)

def hierarchical_clustering(G, n_clusters):
    n_vertices=G.number_of_nodes()
    clusters = [[i] for i in range(n_vertices)]
    distance_matrix = calculate_distance_matrix(G)

    while len(clusters) > n_clusters:
        i, j = find_closest_clusters(distance_matrix)

        new_cluster = clusters[i] + clusters[j]
        clusters.append(new_cluster)

        clusters.pop(max(i, j))
        clusters.pop(min(i, j))

        new_distances = []
        for k in range(len(distance_matrix)):
            if k != i and k != j:
                new_dist = min(distance_matrix[k, i], distance_matrix[k, j])
                new_distances.append(new_dist)
        new_distances.append(0.0)  

        distance_matrix = np.delete(distance_matrix, [max(i, j), min(i, j)], axis=0)
        distance_matrix = np.delete(distance_matrix, [max(i, j), min(i, j)], axis=1)
        new_distances = np.array(new_distances)
        distance_matrix = np.vstack((distance_matrix, new_distances[:-1]))
        distance_matrix = np.column_stack((distance_matrix, new_distances))

    return clusters

# Example usage



In [24]:
pi = np.array([[0.1,1,0.1,0.1],[1,0.1,1,0.1],[0.1,1,0.1,1],[0.1,0.1,1,0.1]])
priors = np.array([0.1, 0.4, 0.4, 0.1])
G=generator.generate(20, pi, priors)

print(G.number_of_edges())
clusters=hierarchical_clustering(G,4)
print(clusters)

97
[[0], [1], [19], [16, 8, 6, 2, 4, 17, 9, 7, 3, 5, 18, 10, 13, 11, 14, 12, 15]]


/var/folders/kk/m7r241y560174wvw5cvj2dfm0000gn/T/ipykernel_50443/1223222977.py:6: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A= nx.adjacency_matrix(G).toarray()
